<a href="https://colab.research.google.com/github/lut-ful/NLP-healthcare/blob/main/category_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from datasets import Dataset, DatasetDict,load_from_disk,load_dataset

In [2]:
dataset_path = "medal_dataset"
# dataset = load_from_disk(dataset_path)

dataset = load_dataset("lutful2004/MeDAL-dataset")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/608 [00:00<?, ?B/s]

data/train-00000-of-00008.parquet:   0%|          | 0.00/254M [00:00<?, ?B/s]

data/train-00001-of-00008.parquet:   0%|          | 0.00/254M [00:00<?, ?B/s]

data/train-00002-of-00008.parquet:   0%|          | 0.00/254M [00:00<?, ?B/s]

...

data/valid-00001-of-00003.parquet:   0%|          | 0.00/226M [00:00<?, ?B/s]

data/valid-00002-of-00003.parquet:   0%|          | 0.00/226M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3000000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

In [3]:
from datasets import ClassLabel

# get unique labels
unique_labels = set()
for split in dataset.keys():
    # Convert to list first to make it hashable
    unique_labels.update(dataset[split]["LABEL"])
unique_labels = sorted(unique_labels)

# create ClassLabel feature
label_class = ClassLabel(num_classes=len(unique_labels), names=unique_labels)


In [4]:
# cast the LABEL column
dataset = dataset.cast_column("LABEL", label_class)

# now you can do stratified sampling
sampled = DatasetDict({
    "train": dataset["train"].train_test_split(
        train_size=0.1, stratify_by_column="LABEL", seed=42
    )["train"],
    "test": dataset["test"].train_test_split(
        train_size=0.1, stratify_by_column="LABEL", seed=42
    )["train"],
    "valid": dataset["valid"].train_test_split(
        train_size=0.1, stratify_by_column="LABEL", seed=42
    )["train"],
})

print(sampled)


Casting the dataset:   0%|          | 0/3000000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1000000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['ABSTRACT_ID', 'TEXT', 'LOCATION', 'LABEL'],
        num_rows: 300000
    })
    test: Dataset({
        features: ['ABSTRACT_ID', 'TEXT', 'LOCATION', 'LABEL'],
        num_rows: 100000
    })
    valid: Dataset({
        features: ['ABSTRACT_ID', 'TEXT', 'LOCATION', 'LABEL'],
        num_rows: 100000
    })
})


In [6]:
sampled

DatasetDict({
    train: Dataset({
        features: ['ABSTRACT_ID', 'TEXT', 'LOCATION', 'LABEL'],
        num_rows: 300000
    })
    test: Dataset({
        features: ['ABSTRACT_ID', 'TEXT', 'LOCATION', 'LABEL'],
        num_rows: 100000
    })
    valid: Dataset({
        features: ['ABSTRACT_ID', 'TEXT', 'LOCATION', 'LABEL'],
        num_rows: 100000
    })
})

In [8]:
from huggingface_hub import notebook_login

# # Create a smaller version of the dataset for easier upload
# small_medal_dataset = DatasetDict({
#     "train": train_dataset.select(range(10000)),
#     "test": test_dataset.select(range(1000)),
#     "valid": valid_dataset.select(range(1000))
# })

# Import required libraries

# Login to Huggingface Hub
notebook_login()

# Push to hub
sampled.push_to_hub(
    "lutful2004/MeDAL-dataset-small",
    private=False,  # Set to True if you want a private dataset
    token=None  # Will use the token from notebook_login
)

print("Dataset uploaded successfully to Huggingface Hub")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/300 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :  12%|#2        | 25.1MB /  202MB            

...

Creating parquet from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :  49%|####9     | 33.6MB / 68.2MB            

Dataset uploaded successfully to Huggingface Hub
